In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import tqdm

import matplotlib.cm as cm
from matplotlib import gridspec
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
from scipy import exp
from scipy.optimize import curve_fit

import pandas as pd
import os
import shutil
import cv2

import glob2

from numba import jit
import tables
from scipy.io import loadmat
import scipy
import h5py
import hdf5storage
from sklearn.metrics.pairwise import euclidean_distances
import matplotlib.cm as cm

from utility_simulated_drift import (make_shifted_templates,
                                      make_default_template,
                                      draw_template,
                                      gaus,
                                      make_drift_template,
                                      shift_templates_all,
                                      WaveForms,
                                      generate_poisson_uniform_firingrate2,
                                      select_ground_truth_units,
                                      visualize_drift,
                                      generate_synthetic_data,
                                      visualize_traces,
                                      make_full_data)

colors = [
'black','grey','brown','slategrey',    
'orange','firebrick','lawngreen','dodgerblue','crimson','orchid','slateblue',
'darkgreen','darkorange','indianred','darkviolet','deepskyblue','greenyellow',
'peru','cadetblue','forestgreen','slategrey','lightsteelblue','rebeccapurple',
'darkmagenta','yellow','hotpink']
clrs= colors


Autosaving every 180 seconds


In [3]:
# select working directory location
root_dir = '/media/cat/4TBSSD/data/synthetic/run31_monotonic_1.5chans_100units/'

ctr=0
while True:
    try:
        os.mkdir(root_dir)
        break
    except:
        root_dir = root_dir[:-1]+"_"+str(ctr)+'/'
        print ("Directory exists... appending another val: ", root_dir)
        

# set radius of channels to consider from maximum channel outwards (in micrometers)
radius = 200

# set data information
sample_rate = 30000
rec_len_sec = 300  # seconds

# select ground truth templates to be used
#fname_templates = '/media/cat/12TB/dbox/Dropbox/code/neuropixels/allen_inst_2ndrec/templates_reloaded.npy'
#geom = np.loadtxt('/home/cat/p1_g0_t0.imec0.ap_geom_384chans.txt')
fname_templates = '/media/cat/4TBSSD/data/synthetic/allen_inst_2ndrec/templates_reloaded.npy'
geom = np.loadtxt('/media/cat/4TBSSD/data/synthetic/p1_g0_t0.imec0.ap_geom.txt')


# select largest X templates to be used for injection
n_templates = 100
(units, templates_ground_truth) = select_ground_truth_units(root_dir, 
                                                            fname_templates,
                                                            n_templates,
                                                            geom)

print ("templates ground truth: ", templates_ground_truth.shape)

Directory exists... appending another val:  /media/cat/4TBSSD/data/synthetic/run31_monotonic_1.5chans_100units_0/
Total templates avialable;  (575, 101, 384)
Total units selected:  100 , ids:  [420 339 421 348 473 418  62 362 565 239] ...
templates ground truth:  (100, 101, 384)


In [4]:
# visualize the desired shifts on a particular unit
#shifts = np.array([-2, -1.5, -1.0, -0.5, 0])
#shifts = np.arange(0, 1., 0.05)
#shifts = np.hstack((shifts, np.array([0,1.6, 2.3])))

drift_type = 'monotonic' # random_walk, or real
if drift_type == 'monotonic':
    shifts = np.arange(0, 1.5, 0.1)
    #shifts = np.array([-2.0, -1.99, 1.99, 2.0])

elif drift_type == 'random_walk':
    # random walk 
    inter_chunk_time = 2
    shifts = np.random.normal(0,1,size=rec_len_sec//inter_chunk_time)*.05
    print ("shifts: ", shifts)
    plt.plot(np.cumsum(shifts))
    shifts = np.cumsum(shifts)

elif drift_type == 'real':
    shifts = np.load('/media/cat/4TBSSD/data/olivier_drift/drift_probe01.npy').squeeze()

    # clip
    shifts = shifts[:rec_len_sec]
    
    # divide by 46. which is the micron -> channels conversion
    shifts /= 46.
    print (shifts.shape)
    
    
print (shifts)

np.savetxt(root_dir+'/ground_truth/shifts.txt', shifts)

# visualize drifted template for any particular unit 
if False:
    unit_id = 50
    visualize_drift(shifts, 
                    geom, 
                    templates_ground_truth, 
                    unit_id,
                    radius)


[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.  1.1 1.2 1.3 1.4]


In [5]:
# generate synthetic drift data
rec_length_sample_times = rec_len_sec * sample_rate
firingrate_min = 1
firingrate_max = 3

# generate synthetic data
(data_synthetic, windows, spike_train) = generate_synthetic_data(root_dir,
                                         rec_len_sec, 
                                         sample_rate, 
                                         shifts, 
                                         units, 
                                         templates_ground_truth,
                                         radius,
                                         geom,
                                         firingrate_min,
                                         firingrate_max)


print (data_synthetic.shape)

(9000000, 384)
[      0.          642857.14285714 1285714.28571429 1928571.42857143
 2571428.57142857 3214285.71428571 3857142.85714286 4500000.
 5142857.14285714 5785714.28571429 6428571.42857143 7071428.57142857
 7714285.71428571 8357142.85714286 9000000.        ]
window:  [0.0, 642857.1428571428] , shift:  0.0  (inter channel units)
 inserting unit:  0 [ 52520  72166  96238 140890 155272 181559 247338 285132 286130 315317] [0.99268221 1.0181501  1.08089349 0.96153886 0.92109244 1.06462593
 0.92831805 1.02322853 0.90576606 0.95658734]
 inserting unit:  50 [ 3491 12258 22819 28511 36315 37382 43360 61935 61986 62517] [0.98921227 1.08402633 1.02883652 0.90716212 0.92423368 0.86225904
 1.08925757 0.9150943  0.89985094 0.90794877]
************


window:  [642857.1428571428, 1285714.2857142857] , shift:  0.1  (inter channel units)
 inserting unit:  0 [705414 719354 745511 752303 808068 837768 871537 874976 881854 957818] [1.02622075 1.00429978 0.92092413 0.9890347  1.09674602 0.9442547
 0

In [6]:
# SAVE DRIFT TIME SERIES
drift = np.zeros((data_synthetic.shape[0]),'float32')
for k in range(len(windows)):
    drift[int(windows[k][0]):int(windows[k][1])]=shifts[k]

fname = root_dir+'/ground_truth/drift.npy'
np.save(fname, drift)


In [7]:
# # visualize the final traces
# visualize_traces(data_synthetic, 0, 5000)


In [8]:
# Make a correlated data chunk of 10 seconds 
# longer takes too long of time; shorter will add more boundary artifcats to recording
time = 10*sample_rate
print ("length of correlated noise snipit: ", time)

fname = root_dir+'correlated_noise.npy'
fname = '/media/cat/4TBSSD/data/synthetic/run18/correlated_noise.npy'
# use ground_truth_templates to make noise
if os.path.exists(fname)==False:
    temps = templates_ground_truth.copy()
    WF = WaveForms(temps.transpose(0,2,1))
    correlated_noise = WF.generate_correlated_noise_add(time)

    print ("DONE MAKING CORRELATED NOISE for 10 sec chunk")
    
    np.save(fname, correlated_noise)
else:
    correlated_noise = np.load(fname)

length of correlated noise snipit:  300000


In [9]:
# blend correlated noise and white noise with injected spike dataset
data_sum = make_full_data(data_synthetic, rec_length_sample_times, 
                          sample_rate, root_dir,
                          correlated_noise)


white noise:  (300000, 384)
time chunk:  0  to  300000
time chunk:  300000  to  600000
time chunk:  600000  to  900000
time chunk:  900000  to  1200000
time chunk:  1200000  to  1500000
time chunk:  1500000  to  1800000
time chunk:  1800000  to  2100000
time chunk:  2100000  to  2400000
time chunk:  2400000  to  2700000
time chunk:  2700000  to  3000000
time chunk:  3000000  to  3300000
time chunk:  3300000  to  3600000
time chunk:  3600000  to  3900000
time chunk:  3900000  to  4200000
time chunk:  4200000  to  4500000
time chunk:  4500000  to  4800000
time chunk:  4800000  to  5100000
time chunk:  5100000  to  5400000
time chunk:  5400000  to  5700000
time chunk:  5700000  to  6000000
time chunk:  6000000  to  6300000
time chunk:  6300000  to  6600000
time chunk:  6600000  to  6900000
time chunk:  6900000  to  7200000
time chunk:  7200000  to  7500000
time chunk:  7500000  to  7800000
time chunk:  7800000  to  8100000
time chunk:  8100000  to  8400000
time chunk:  8400000  to  870000

In [12]:
# # visualize first 10000 time points;
# visualize_traces(data_sum, 0, 10000)


In [9]:
# import numpy as np
# import h5py

# #mystuff = np.random.rand(10,30)

# f = h5py.File(root_dir+'/data.h5', 'w')
# f['rawdata'] = data_sum
# f['geometry'] = geom
# f['drift'] = drift
# f['spike_times'] = spike_train[:,0]
# f['neuron_ids'] = spike_train[:,1]
# f.close()

# REMAKE / UPDATE HDF5 FILE FROM PREVIOUSL SAVED DATA


In [13]:
import numpy as np
import h5py

if False:
    #root_dir = '/media/cat/4TBSSD/data/synthetic/run28_olivier_real_drift/'
    geom = np.loadtxt(root_dir + '/geom_384chans.txt')
    drift = np.load(root_dir + '/ground_truth/drift.npy')
    spike_train = np.load(root_dir + '/ground_truth/spike_train_ground_truth.npy')
    templates = np.load(root_dir + '/ground_truth/templates_ground_truth.npy')
    
    # add new fields
    try:
        reorder = np.load(root_dir + '/tmp/preprocess/reorder.npy')
        spike_train_detect = np.load(root_dir + '/tmp/block_1/detect/spike_index.npy')
    except:
        print ("DATA not sorted yet...")
    
    # make max channel information
    max_chans = templates.ptp(1).argmax(1)
    channel_locations = spike_train[:,1].copy()
    for k in np.unique(spike_train[:,1]):
        idx = np.where(spike_train[:,1]==k)[0]
        channel_locations[idx]=max_chans[k]

    # Make HDF5 file

    data_sum = np.fromfile(root_dir + '/data_int16.bin','int16')

    f2 = h5py.File(root_dir+'/data.h5', 'w')
    f2['rawdata'] = data_sum
    f2['geometry'] = geom
    f2['ground_truth_drift'] = drift
    f2['ground_truth_spike_times'] = spike_train[:,0]
    f2['ground_truth_neuron_ids'] = spike_train[:,1]
    f2['kilosort2_reorder'] = reorder
    f2['detected_spike_times'] = spike_train_detect[:,0]
    f2['detected_neuron_channels'] = spike_train_detect[:,1]
    f2['templates'] = templates   
    f2['channel_locations'] = channel_locations

    f2.close()

DATA not sorted yet...


NameError: name 'reorder' is not defined

# CHECK RESULTS AFTER KS2 REORDERING

In [24]:
# TEST KS2 RESULTS
import numpy as np
import h5py

sample_rate = 30000

root_dir = '/media/cat/4TBSSD/data/synthetic/run27_monotonic/'
#root_dir = '/media/cat/4TBSSD/data/synthetic/run26_random_walk/'

reorder = np.load(root_dir + '/tmp/preprocess/reorder.npy')
#print (reorder)

fig=plt.figure()
ax1=plt.subplot(1,1,1)

#mystuff = np.random.rand(10,30)
if False:
    fname = root_dir + '/run27_monotonic.h5'
    f = h5py.File(fname, 'r')
    drift = np.float32(f['drift']).copy()
else:
    drift = np.load(root_dir + '/ground_truth/drift.npy')
    


    
x= np.arange(drift.shape[0])/sample_rate
plt.plot(x,drift, c='black', label="Original drift")

# compute ideal drift
idx = np.argsort(drift)#[::-1]
plt.plot(x,drift[idx],c='blue',label='Ideal monotonic reordering')


# do KS2 reorder
reorder = reorder[::-1]
chunk = 1 #* sample_rate #seconds
diffs = []

for k in range(reorder.shape[0]):
    
    # load KS2 reordered chunk value starts and ends
    ks2_trace = reorder[k]*chunk*sample_rate
    start = int(ks2_trace)
    end = int(start + chunk*sample_rate)
    
    # grab chunk from ground truth drift trace
    ks2_trace = drift[start:end]
    
    # make time series offset to beginning of chunk; 
    x = np.arange(chunk*sample_rate)+k*chunk*sample_rate
    plt.plot(x/sample_rate, ks2_trace,c='red')

    # ground truth drift; ideal reordering
    gt_trace = drift[idx][k*sample_rate:(k+chunk)*sample_rate]
    #temp2 = temp
    print (k, ks2_trace, gt_trace, ks2_trace.shape, gt_trace.shape)
    #diff = np.abs(temp1-temp2).mean(0)
    diff = np.sqrt(((ks2_trace-gt_trace)**2).mean())
           #np.sqrt(((predictions - targets) ** 2).mean())
    diffs.append(diff)
    
print (diffs)
plt.plot(x/sample_rate, temp,c='red', label="KS2 monotonic reorder")
plt.legend(fontsize=12, loc=2)
plt.suptitle(root_dir,fontsize=20)
plt.ylabel("Drift in Channels ", fontsize=20)
plt.xlabel("Time (sec) ", fontsize=20)

#ax2=plt.subplot(1,2,2)
plt.title("Distribution of (L2) KS2 errors; mean = "+str(round(np.mean(diffs),4)))
#bin_width = 0.05
#y = np.histogram(diffs,np.arange(-1,1,bin_width))
#plt.plot(y[1][:-1]+bin_width,y[0])

plt.show()

0 [-0.4 -0.4 -0.4 ... -0.4 -0.4 -0.4] [-0.4 -0.4 -0.4 ... -0.4 -0.4 -0.4] (30000,) (30000,)
1 [-0.39 -0.39 -0.39 ... -0.39 -0.39 -0.39] [-0.4 -0.4 -0.4 ... -0.4 -0.4 -0.4] (30000,) (30000,)
2 [-0.38 -0.38 -0.38 ... -0.38 -0.38 -0.38] [-0.4 -0.4 -0.4 ... -0.4 -0.4 -0.4] (30000,) (30000,)
3 [-0.36 -0.36 -0.36 ... -0.36 -0.36 -0.36] [-0.4 -0.4 -0.4 ... -0.4 -0.4 -0.4] (30000,) (30000,)
4 [-0.37 -0.37 -0.37 ... -0.37 -0.37 -0.37] [-0.4 -0.4 -0.4 ... -0.4 -0.4 -0.4] (30000,) (30000,)
5 [-0.34 -0.34 -0.34 ... -0.33 -0.33 -0.33] [-0.4  -0.4  -0.4  ... -0.39 -0.39 -0.39] (30000,) (30000,)
6 [-0.36 -0.36 -0.36 ... -0.36 -0.36 -0.36] [-0.39 -0.39 -0.39 ... -0.39 -0.39 -0.39] (30000,) (30000,)
7 [-0.31 -0.31 -0.31 ... -0.31 -0.31 -0.31] [-0.39 -0.39 -0.39 ... -0.39 -0.39 -0.39] (30000,) (30000,)
8 [-0.4 -0.4 -0.4 ... -0.4 -0.4 -0.4] [-0.39 -0.39 -0.39 ... -0.39 -0.39 -0.39] (30000,) (30000,)
9 [-0.25 -0.25 -0.25 ... -0.25 -0.25 -0.25] [-0.39 -0.39 -0.39 ... -0.39 -0.39 -0.39] (30000,) (30000,)
10

80 [-0.17 -0.17 -0.17 ... -0.17 -0.17 -0.17] [-0.25 -0.25 -0.25 ... -0.25 -0.25 -0.25] (30000,) (30000,)
81 [-0.2 -0.2 -0.2 ... -0.2 -0.2 -0.2] [-0.25 -0.25 -0.25 ... -0.24 -0.24 -0.24] (30000,) (30000,)
82 [-0.25 -0.25 -0.25 ... -0.24 -0.24 -0.24] [-0.24 -0.24 -0.24 ... -0.24 -0.24 -0.24] (30000,) (30000,)
83 [-0.24 -0.24 -0.24 ... -0.24 -0.24 -0.24] [-0.24 -0.24 -0.24 ... -0.24 -0.24 -0.24] (30000,) (30000,)
84 [-0.21 -0.21 -0.21 ... -0.21 -0.21 -0.21] [-0.24 -0.24 -0.24 ... -0.24 -0.24 -0.24] (30000,) (30000,)
85 [-0.22 -0.22 -0.22 ... -0.22 -0.22 -0.22] [-0.24 -0.24 -0.24 ... -0.24 -0.24 -0.24] (30000,) (30000,)
86 [-0.26 -0.26 -0.26 ... -0.26 -0.26 -0.26] [-0.24 -0.24 -0.24 ... -0.23 -0.23 -0.23] (30000,) (30000,)
87 [-0.22 -0.22 -0.22 ... -0.22 -0.22 -0.22] [-0.23 -0.23 -0.23 ... -0.23 -0.23 -0.23] (30000,) (30000,)
88 [-0.27 -0.27 -0.27 ... -0.27 -0.27 -0.27] [-0.23 -0.23 -0.23 ... -0.23 -0.23 -0.23] (30000,) (30000,)
89 [-0.21 -0.21 -0.21 ... -0.21 -0.21 -0.21] [-0.23 -0.23 -0.

160 [-0.01 -0.01 -0.01 ... -0.01 -0.01 -0.01] [-0.09 -0.09 -0.09 ... -0.09 -0.09 -0.09] (30000,) (30000,)
161 [-0.08 -0.08 -0.08 ... -0.07 -0.07 -0.07] [-0.09 -0.09 -0.09 ... -0.09 -0.09 -0.09] (30000,) (30000,)
162 [-0.1 -0.1 -0.1 ... -0.1 -0.1 -0.1] [-0.09 -0.09 -0.09 ... -0.08 -0.08 -0.08] (30000,) (30000,)
163 [-0.07 -0.07 -0.07 ... -0.07 -0.07 -0.07] [-0.08 -0.08 -0.08 ... -0.08 -0.08 -0.08] (30000,) (30000,)
164 [-0.09 -0.09 -0.09 ... -0.09 -0.09 -0.09] [-0.08 -0.08 -0.08 ... -0.08 -0.08 -0.08] (30000,) (30000,)
165 [-0.19 -0.19 -0.19 ... -0.19 -0.19 -0.19] [-0.08 -0.08 -0.08 ... -0.08 -0.08 -0.08] (30000,) (30000,)
166 [-0.03 -0.03 -0.03 ... -0.03 -0.03 -0.03] [-0.08 -0.08 -0.08 ... -0.08 -0.08 -0.08] (30000,) (30000,)
167 [-0.15 -0.15 -0.15 ... -0.15 -0.15 -0.15] [-0.08 -0.08 -0.08 ... -0.07 -0.07 -0.07] (30000,) (30000,)
168 [-0.14 -0.14 -0.14 ... -0.13 -0.13 -0.13] [-0.07 -0.07 -0.07 ... -0.07 -0.07 -0.07] (30000,) (30000,)
169 [0.01 0.01 0.01 ... 0.01 0.01 0.01] [-0.07 -0.07

240 [0.08 0.08 0.08 ... 0.08 0.08 0.08] [0.07 0.07 0.07 ... 0.07 0.07 0.07] (30000,) (30000,)
241 [0.07 0.07 0.07 ... 0.07 0.07 0.07] [0.07 0.07 0.07 ... 0.07 0.07 0.07] (30000,) (30000,)
242 [0.06 0.06 0.06 ... 0.06 0.06 0.06] [0.07 0.07 0.07 ... 0.07 0.07 0.07] (30000,) (30000,)
243 [-0.04 -0.04 -0.04 ... -0.04 -0.04 -0.04] [0.07 0.07 0.07 ... 0.08 0.08 0.08] (30000,) (30000,)
244 [0.09 0.09 0.09 ... 0.09 0.09 0.09] [0.08 0.08 0.08 ... 0.08 0.08 0.08] (30000,) (30000,)
245 [0.07 0.07 0.07 ... 0.07 0.07 0.07] [0.08 0.08 0.08 ... 0.08 0.08 0.08] (30000,) (30000,)
246 [0.09 0.09 0.09 ... 0.09 0.09 0.09] [0.08 0.08 0.08 ... 0.08 0.08 0.08] (30000,) (30000,)
247 [0.08 0.08 0.08 ... 0.09 0.09 0.09] [0.08 0.08 0.08 ... 0.08 0.08 0.08] (30000,) (30000,)
248 [0.11 0.11 0.11 ... 0.11 0.11 0.11] [0.08 0.08 0.08 ... 0.09 0.09 0.09] (30000,) (30000,)
249 [0.06 0.06 0.06 ... 0.07 0.07 0.07] [0.09 0.09 0.09 ... 0.09 0.09 0.09] (30000,) (30000,)
250 [0.08 0.08 0.08 ... 0.08 0.08 0.08] [0.09 0.09 0.0

330 [0.29 0.29 0.29 ... 0.29 0.29 0.29] [0.25 0.25 0.25 ... 0.25 0.25 0.25] (30000,) (30000,)
331 [0.28 0.28 0.28 ... 0.29 0.29 0.29] [0.25 0.25 0.25 ... 0.25 0.25 0.25] (30000,) (30000,)
332 [0.34 0.34 0.34 ... 0.34 0.34 0.34] [0.25 0.25 0.25 ... 0.25 0.25 0.25] (30000,) (30000,)
333 [0.2 0.2 0.2 ... 0.2 0.2 0.2] [0.25 0.25 0.25 ... 0.25 0.25 0.25] (30000,) (30000,)
334 [0.28 0.28 0.28 ... 0.28 0.28 0.28] [0.25 0.25 0.25 ... 0.26 0.26 0.26] (30000,) (30000,)
335 [0.22 0.22 0.22 ... 0.22 0.22 0.22] [0.26 0.26 0.26 ... 0.26 0.26 0.26] (30000,) (30000,)
336 [0.23 0.23 0.23 ... 0.24 0.24 0.24] [0.26 0.26 0.26 ... 0.26 0.26 0.26] (30000,) (30000,)
337 [0.23 0.23 0.23 ... 0.23 0.23 0.23] [0.26 0.26 0.26 ... 0.26 0.26 0.26] (30000,) (30000,)
338 [0.32 0.32 0.32 ... 0.32 0.32 0.32] [0.26 0.26 0.26 ... 0.26 0.26 0.26] (30000,) (30000,)
339 [0.24 0.24 0.24 ... 0.25 0.25 0.25] [0.26 0.26 0.26 ... 0.27 0.27 0.27] (30000,) (30000,)
340 [0.24 0.24 0.24 ... 0.24 0.24 0.24] [0.27 0.27 0.27 ... 0.27 0

In [2]:
# Load previously saved dataset
root_dir = '/media/cat/4TBSSD/data/synthetic/run27_monotonic/'

f = h5py.File(root_dir+'/data.h5', 'r')
print (f.keys())
drift = np.float32(f.get('drift'))
data_sum = np.array(f.get('rawdata'))
geom = np.array(f.get('geometry'))
spike_times = np.array(f.get('spike_times'))
spike_ids = np.array(f.get('neuron_ids'))
f.close()

print (data_sum.shape)



<KeysViewHDF5 ['drift', 'geometry', 'neuron_ids', 'rawdata', 'spike_times', 'templates']>
(4608000000,)


In [3]:
# add new fields
reorder = np.load(root_dir + '/tmp/preprocess/reorder.npy')
spike_train_detect = np.load(root_dir + '/tmp/block_1/detect/spike_index.npy')


In [ ]:
# add new fileds

In [4]:
f2 = h5py.File(root_dir+'/data_updated.h5', 'w')
f2['rawdata'] = data_sum
f2['geometry'] = geom
f2['ground_truth_drift'] = drift
f2['ground_truth_spike_times'] = spike_times
f2['ground_truth_neuron_ids'] = spike_ids
f2['kilosort2_reorder'] = reorder
f2['detected_spike_times'] = spike_train_detect[:,0]
f2['detected_neuron_channels'] = spike_train_detect[:,1]
f2.close()

In [8]:
events = np.load('/media/cat/4TBSSD/data/synthetic/run25_random_walk/tmp/block_1/detect/spike_index.npy')


In [5]:
data = np.load('/media/cat/4TBSSD/data/olivier_drift/drift_probe00.npy')
plt.plot(data/46.)
plt.show()

In [22]:
# add channels
root_dir = '/media/cat/4TBSSD/data/synthetic/run27_monotonic/'
spike_train = np.load(root_dir + '/ground_truth/spike_train_ground_truth.npy')
templates = np.load(root_dir +'/ground_truth/templates_ground_truth.npy')
print (templates.shape)

max_chans = templates.ptp(1).argmax(1)
channel_locations = spike_train[:,1].copy()
for k in np.unique(spike_train[:,1]):
    idx = np.where(spike_train[:,1]==k)[0]
    channel_locations[idx]=max_chans[k]
    
print (channel_locations[:40])
print (spike_train[:40,1])
f2['channel_locations'] = channel_locations


(100, 101, 384)
[251 251 251 251 251 251 251 251 251 251 251 251 251 251 251 251 251 251
 251 182 182 182 182 182 182 182 182 182 182 182 182 182 182 182 182 182
 182 182 182 182]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1]


In [23]:
f2 = h5py.File(root_dir+'/data_updated_channels.h5', 'w')

f2['channel_locations'] = channel_locations
f2.close()

In [25]:
print (spike_train.shape)

(123858, 2)


In [12]:
pos = np.load('/media/cat/4TBSSD/Downloads/Copy of Copy of manip.positions.npy')/46.
data = np.load('/media/cat/4TBSSD/Downloads/Copy of Copy of manip.timestamps_p2.npy')
ax=plt.subplot(121)
plt.plot(data, pos)
plt.title("NP1 probe")
plt.ylabel("Drift (channel units)")
plt.xlabel("Time (sec)")

data2 = np.load('/media/cat/4TBSSD/Downloads/Copy of Copy of manip.timestamps_p1.npy')
ax=plt.subplot(122)
plt.title("NP2 probe")
plt.xlabel("Time (sec)")
plt.plot(data2, pos,c='black')
plt.show()